In [6]:
tsize = 0.8
K = 300

In [2]:
import pandas as pd
df = pd.read_csv('Whole_dataset1.txt')

### Changing the time-format

In [3]:
import scipy as np
Date = np.array(df.Date); time = np.empty(Date.shape[0])
for i in range(Date.shape[0]):
    time[i] = int(''.join(Date[i].split('-')))
del Date
df['Date'] = time; del time

### Mapping to maintain continuity

In [4]:
df = df.sort_values(by='CustomerID')
import gc; gc.collect()
UID = np.array(df.CustomerID)
count = 0; flag=UID[0]
for i in range(UID.shape[0]):
    if(flag==UID[i]):
        UID[i]=count
    else:
        count += 1
        flag=UID[i]
        UID[i]=count
df['CustomerID'] = UID; del UID
df = df.sort_values(by='MovieID')
MID = np.array(df.MovieID)
count = 0; flag=MID[0]
for i in range(MID.shape[0]):
    if(flag==MID[i]):
        MID[i]=count
    else:
        count += 1
        flag=MID[i]
        MID[i]=count
df['MovieID'] = MID; del MID

    The data is mapped so that the sparse matrix can be made easily.

In [12]:
MID = np.array(df.MovieID)
UID = np.array(df.CustomerID)
Score = np.array(df.Score)
df = df.sample(frac=1)
#del df
MID.max(), UID.max()

(4498, 470757)

In [13]:
ts_data = np.copy(Score)
l, = Score.shape
for i in range(int(tsize*l),l):
    Score[i]=0
Score[-10:], ts_data[-10:]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([4, 3, 3, 4, 4, 3, 3, 3, 5, 4]))

In [14]:
from scipy.sparse import coo_matrix
SPM = coo_matrix((Score,(UID,MID)),shape=(UID.max()+1,MID.max()+1))
SPM = SPM.asfptype()
SPM

<470758x4499 sparse matrix of type '<class 'numpy.float64'>'
	with 24053764 stored elements in COOrdinate format>

In [ ]:
from scipy.sparse.linalg import svds
U,sig,VT = svds(SPM,k=K)
U.shape, sig.shape, VT.shape

In [ ]:
# test data
sig = np.diag(sig)
Ui = np.dot(U,sig)
e=0; count = 0
for i in range(int(tsize*l),l):
    count += 1
    r = UID[i]; c = MID[i]
    R = ts_data[i]
    Rh = np.dot(Ui[r,:],VT[:,c])
    e += (R-Rh)**2
e = e/count
e**0.5

In [ ]:
# train data
e=0; count = 0
for i in range(int(tsize*l)):
    count += 1
    r = UID[i]; c = MID[i]
    R = ts_data[i]
    Rh = np.dot(Ui[r,:],VT[:,c])
    e += (R-Rh)**2
e = e/count
e**0.5